In [ ]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [ ]:
# Завантажуємо датасет
df = pd.read_csv('../data/adult.csv', index_col=0)

# Перевіряємо структуру даних
print("Розмір датасету:", df.shape)
print("\nПерші 5 рядків:")
print(df.head())

# Перевіряємо на наявність знаків питання
print("\nПеревірка на знаки питання в кожній колонці:")
for col in df.columns:
    question_count = (df[col] == '?').sum()
    if question_count > 0:
        print(f"{col}: {question_count} знаків питання")

# Видаляємо рядки з знаками питання
print(f"\nРозмір до очищення: {df.shape}")
df_clean = df[~df.isin(['?']).any(axis=1)]
print(f"Розмір після очищення: {df_clean.shape}")

# Перевіряємо відповідність між salary та salary K$
print("\nПеревірка відповідності salary та salary K$:")
print("Унікальні значення salary:", df_clean['salary'].unique())
print("Унікальні значення salary K$:", df_clean['salary K$'].unique())

# Перевіряємо логічну відповідність
mismatch = df_clean[
    ((df_clean['salary'] == '>50K') & (df_clean['salary K$'] <= 50)) |
    ((df_clean['salary'] == '<=50K') & (df_clean['salary K$'] > 50))
]
print(f"Кількість невідповідностей: {len(mismatch)}")

# Зберігаємо очищений датасет
df = df_clean.copy()
print(f"\nФінальний розмір датасету: {df.shape}")

# Task 1
Print the count of men and women in the dataset.

In [1]:
# Підраховуємо кількість чоловіків та жінок
sex_counts = df['sex'].value_counts()
print("Кількість людей за статтю:")
print(sex_counts)
print(f"\nЧоловіків: {sex_counts['Male']}")
print(f"Жінок: {sex_counts['Female']}")

# Task 2
Find the average age of men in dataset

In [ ]:
# Знаходимо середній вік чоловіків
men_avg_age = df[df['sex'] == 'Male']['age'].mean()
print(f"Середній вік чоловіків: {men_avg_age:.2f} років")

# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
# Отримуємо відсоток людей з Польщі
poland_count = (df['native-country'] == 'Poland').sum()
total_count = len(df)
poland_percentage = (poland_count / total_count) * 100

print(f"Кількість людей з Польщі: {poland_count}")
print(f"Загальна кількість людей: {total_count}")
print(f"Відсоток людей з Польщі: {poland_percentage:.2f}%")

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [2]:
# Статистика віку для людей з зарплатою > 50K
high_salary = df[df['salary'] == '>50K']['age']
high_salary_mean = high_salary.mean()
high_salary_std = high_salary.std()

print("Люди з зарплатою > 50K:")
print(f"Середній вік: {high_salary_mean:.2f} років")
print(f"Стандартне відхилення: {high_salary_std:.2f} років")

# Статистика віку для людей з зарплатою <= 50K
low_salary = df[df['salary'] == '<=50K']['age']
low_salary_mean = low_salary.mean()
low_salary_std = low_salary.std()

print("\nЛюди з зарплатою <= 50K:")
print(f"Середній вік: {low_salary_mean:.2f} років")
print(f"Стандартне відхилення: {low_salary_std:.2f} років")

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [3]:
# Визначаємо вищу освіту
higher_education = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']

# Люди без вищої освіти з зарплатою > 50K
no_higher_ed_high_salary = df[
    (~df['education'].isin(higher_education)) & 
    (df['salary'] == '>50K')
]

print(f"Кількість людей без вищої освіти з зарплатою > 50K: {len(no_higher_ed_high_salary)}")

if len(no_higher_ed_high_salary) > 0:
    print("\nТак, є люди без вищої освіти з високою зарплатою!")
    print(f"Це складає {len(no_higher_ed_high_salary)} осіб")
    
    # Показуємо розподіл їх освіти
    print("\nРозподіл за рівнем освіти:")
    print(no_higher_ed_high_salary['education'].value_counts())
else:
    print("Немає людей без вищої освіти з зарплатою > 50K")

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
# Статистика віку за типом освіти
age_by_education = df.groupby('education')['age'].describe()

print("Статистика віку за типом освіти:")
print(age_by_education.round(2))

# Додатково - сортуємо за середнім віком
print("\nТипи освіти відсортовані за середнім віком:")
mean_age_sorted = age_by_education['mean'].sort_values(ascending=False)
print(mean_age_sorted.round(2))

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [5]:
# Фільтруємо тільки чоловіків
men_df = df[df['sex'] == 'Male']

# Визначаємо одружених та неодружених чоловіків
married_men = men_df[men_df['marital-status'].str.startswith('Married')]
non_married_men = men_df[~men_df['marital-status'].str.startswith('Married')]

# Порівнюємо зарплати
print("Одружені чоловіки:")
married_salary_counts = married_men['salary'].value_counts()
print(married_salary_counts)
married_high_salary_pct = (married_salary_counts['>50K'] / len(married_men)) * 100
print(f"Відсоток з високою зарплатою: {married_high_salary_pct:.2f}%")

print("\nНеодружені чоловіки:")
non_married_salary_counts = non_married_men['salary'].value_counts()
print(non_married_salary_counts)
non_married_high_salary_pct = (non_married_salary_counts['>50K'] / len(non_married_men)) * 100
print(f"Відсоток з високою зарплатою: {non_married_high_salary_pct:.2f}%")

print(f"\nВисновок:")
if married_high_salary_pct > non_married_high_salary_pct:
    print("Одружені чоловіки заробляють більше")
else:
    print("Неодружені чоловіки заробляють більше")
    
print(f"Різниця: {abs(married_high_salary_pct - non_married_high_salary_pct):.2f} процентних пунктів")

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [6]:
# Знаходимо максимальну кількість годин на тиждень
max_hours = df['hours-per-week'].max()
print(f"Максимальна кількість годин на тиждень: {max_hours}")

# Підраховуємо скільки людей працює таку ж кількість годин
people_with_max_hours = (df['hours-per-week'] == max_hours).sum()
print(f"Кількість людей, які працюють {max_hours} годин на тиждень: {people_with_max_hours}")

# Додаткова інформація
print(f"\nДодаткова статистика годин роботи:")
hours_stats = df['hours-per-week'].describe()
print(hours_stats)

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [7]:
# Спочатку підготуємо дані для кореляційного аналізу
# Кодуємо категоріальні змінні
df_corr = df.copy()

# Кодуємо salary як числову змінну
df_corr['salary_numeric'] = (df_corr['salary'] == '>50K').astype(int)

# Кодуємо стать
df_corr['sex_numeric'] = (df_corr['sex'] == 'Male').astype(int)

# Кодуємо сімейний стан (одружені = 1)
df_corr['married'] = df_corr['marital-status'].str.startswith('Married').astype(int)

# Відбираємо числові колонки та закодовані категоріальні
numeric_cols = ['age', 'hours-per-week', 'salary K$', 'salary_numeric', 'sex_numeric', 'married']
correlation_matrix = df_corr[numeric_cols].corr()

print("Кореляційна матриця:")
print(correlation_matrix.round(3))

# Знаходимо найбільш корельовані пари (крім діагоналі)
print("\nНайсильніші кореляції:")
correlations = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        corr_value = correlation_matrix.iloc[i, j]
        correlations.append((abs(corr_value), corr_value, col1, col2))

# Сортуємо за абсолютним значенням кореляції
correlations.sort(reverse=True)

for abs_corr, corr, col1, col2 in correlations[:5]:
    print(f"{col1} ↔ {col2}: {corr:.3f}")
    
print("\nОсновні висновки:")
print("1. Зарплата (salary_numeric) найбільше корелює з:")
salary_corr = correlation_matrix['salary_numeric'].abs().sort_values(ascending=False)[1:]  # виключаємо саму себе
for col in salary_corr.head(3).index:
    corr_val = correlation_matrix.loc['salary_numeric', col]
    print(f"   - {col}: {corr_val:.3f}")

print("\n2. Інші важливі зв'язки:")
print(f"   - Вік та одруженість: {correlation_matrix.loc['age', 'married']:.3f}")
print(f"   - Стать та зарплата: {correlation_matrix.loc['sex_numeric', 'salary_numeric']:.3f}")
print(f"   - Години роботи та зарплата: {correlation_matrix.loc['hours-per-week', 'salary_numeric']:.3f}")